# Introduction
To create a genetic linkage mapping, we have to calculate distances between genes. With a substantial amount of genes, this becomes very tedious if done manually. Luckily we can automate the process with these cheeky little scripts $^{[1]}$ . In this study we use (PCR amplified) molecular markers (genotypes a,b) in an arabidopsis plant. Where the amount of markers is 144 and the amount of progeny 162.



$^{[1]}$: This script hasn't implemented the right statistical chi-square test yet due to a bunch of failures, so all data is assumed to be 'good'.

# ~~Loading the data~~  Repairing the data to make it usable
The dataset is scuffed. Data for each marker is smeared over 4 uneven lines in a textfile. A small script should be able to parse this, either into objects, separate files or insert it into a database.

> For every plant in the population of 162 plants, markers are somehow scattered in their dna
I dont ***** know okay? Some parent plants did the thingy and babyplants came out with their genes all over the place. 

In this study we try and find linked genes and map them relative to eachother.

Anyhow.
The first lines of the file are scuffed with some random meta information.

For now, we'll just skip the first 7 lines because we know these are not used in parsing the rest of the file. But first, some variables have to be set.

### Setting globals
Here some variables are globally (meaning accessible in each cell of the script) defined. If you want to test this script with another file

In [2]:
# Globals
MAIN_DATA_FILE = "CvixLerC9.loc"
LEADING_LINES_TO_SKIP = 7
OUTPUT_FILENAME = "output RF matrix of " + MAIN_DATA_FILE + ".txt"

In [3]:
# import re
# regex_object = re.compile("([A-Z0-9.]+) \(a,b\) ; [0-9]+")

### Skipping data 
Now we open the file, skip the first 7 `LEADING_LINES_TO_SKIP`
and start linking the marker observations for each plant as a list in a dictionary with marker ID's being the key.


### Using regex?
A line containing data for the `current_marker` will always have leading spaces. Markers concur with the following regex pattern `([A-Z0-9.]+) \(a,b\) ; [0-9]+` 
> credits to Sanne Schröduer

A syntatically simpler way to extract the marker ID and associated number is to find any lines containing a `;` (semicolon) character and splitting it on `' '` (spaces, of the whitespaceidae)
> Todos 
    - test if this is also true for speed
    - figure out a way to parse the data in a way so that we can just use bits instead of characters (for example, 3 64bytes) ??

## Getting only the right values
The values for a marker are represented with symbols `"a"`, `"b"`, or `"-"`. A line in the file consists of newline characters, space characters and these three symbols of interest. We'll just collect the symbols as we iterate along the lines. Whenever we encounter a new marker-line, the previously collected symbols will be safe in a list linked to the previous marker and a new list will be made to collect the next symbols in.


In [4]:
with open(MAIN_DATA_FILE, 'r') as f:
    for i in range(LEADING_LINES_TO_SKIP):
        # just read the skippable lines and dont do anything with them
        f.readline()
        
    identifier_dict = dict()
    
    for line in f:
        
        # at the end of the data, some more data is displayed, separated by a newline character
        # so we end it here
        if line == "\n":
            break
        
        # The marker identifier and number are contained in lines containing a semicolon
        if ";" in line:
            # isolate the identifier and the number
            l = line.split(' ')
            id_name = l[0]
            number = l[3].replace("\n", "")
            marker_id = f"{id_name}_{number}"
            
            # to make sure a marker doesn't appear twice, throw an error if it does.
            if marker_id in identifier_dict.keys():
                raise ValueError
            
            # if no error occurs, create an empty list as value for this marker.
            else:
                identifier_dict[marker_id] = list()
        
        # if no semicolon is found, this line should consist of data for the current marker
        else:
            # first we have to parse the line to remove whitespaces
            # we'll check if the line complies to the rule "start with a space", at least.
            if line.startswith("  "):
                # now add all characters of this line to the list of the current marker
                identifier_dict[marker_id] += [char for char in line if char in {"a", "b", "-"}]

                
print('here are 5 keys, as an example.')
print(list(identifier_dict.keys())[:5])


                
            
            
            

here are 5 keys, as an example.
['PVV4_1', 'CRY2_2', 'AXR-1_3', 'HH.335C_4', 'DF.162L_5']


## Evaluate parsed data - the amount of genotypes under each marker should be the same

Lets check if the length of each list is the same. Since there are 162 plants in the population, we can also test if the length of each list is equal to 162. 

In [5]:
##### take a random key and determine the size of its data (amount of <genotypes?=(?)>)
##### TODO VERIFY ^

# WRONG ?!
lenn = len(identifier_dict['DF.328C_81'])
assert all([len(l) ==  lenn for l in identifier_dict.values()])
assert all([len(l) == 162 for l in identifier_dict.values()])
print('lengths asserted. All lengths are equal to ', lenn, ".")
print('we can also test if the asserion is working properly in jupyter notebook, to make sure')

assert all([True, False, True])

lengths asserted. All lengths are equal to  162 .
we can also test if the asserion is working properly in jupyter notebook, to make sure


AssertionError: 

# ~~Chi-square to determine integrity of data~~
To test the quality of the data, the chi-square test can be used. < because reason > only one degree of freedom has to be accounted for. 
To determine the validity of our data we can perform the chi-square test. 

\begin{align}
\texttt{Chi Square} = 
\frac{ (\texttt{amount of a's} - \frac{\texttt{total progeny}}{2})^{2} }
 {\frac{\texttt{total progeny}}{2}} + 
\frac{(\texttt{amount of b's} - \frac{\texttt{total progeny}}{2})^{2}}
  {\frac{\texttt{total progeny}}{2}}
\end{align}

In [7]:

# failed chi-square section
markers_to_reject = []

with open('chi_squares voor ' +MAIN_DATA_FILE+ ".txt", 'w') as out:

    out.write(f"marker\tchi-squared\tobserved_a\texpected_a\tobserved_b\texpected_b,total_genotypes\n")
    for k, values in identifier_dict.items():
        total_genotypes = (len(values) - values.count('-'))

        observed_a = values.count('a')
        observed_b = values.count('b')
        expected_a = total_genotypes / 2    # we expect a:b to be 1:3 where b is ab,bb,ba
        expected_b =  total_genotypes / 2
        chi_squared = ((((observed_a - expected_a)**2) / expected_a ) + \
                      (((observed_b - expected_b)**2) / expected_b ))
        # print(observed_a, expected_a, observed_b, expected_b, total_genotypes, chi_squared)
        
        out.write(f"{k}\t{round(chi_squared,2)}\t{observed_a}\t{expected_a}\t{observed_b}\t{expected_b},{total_genotypes}\n")
        # note: total_genotypes gebaseerd op a's en b's, -'s niet meerekenen.

        if chi_squared > 3.841: # ignore this marker.
            markers_to_reject.append((k, chi_squared))
            # print(k,'will be ignored with a chi_squared of',chi_squared)
        else:
            # print(chi_squared)
            pass
            

with open('chi_squares_rejected.txt', 'w') as out:
    for marker, chi_squared in markers_to_reject:
        identifier_dict.pop(marker)
        out.write(f"{marker}\t{chi_squared}\n")
        # print('removed', marker, 'from dataset')


# Sorting data a bit
Now lets alphabetize the data so it will be more readable $^{[2]}$, then, we create a matrix where the distances (in centimorgan) are calculated by comparing their data and counting recominbants, then dividing by the amount of progeny and multiplying by 100. A recombinant is defined by a set of genotype-symbols for two markers in a plant (the occurrence or absence of a marker in the genotype) where ~~any or both~~ one of the symbols is a 'b' and the other an 'a'.

the `recombinant_factor`(RF) is calculated as follows : 
\begin{align}
Recombinant Factor = \frac{N_{recombinants}}{N_{progeny}} * 100 
\end{align}

$^{[2]}$: I did this but then removed it again



# `calculateRF` function

We'll use a function that takes two lists, each holding all symbols representing genotypes for one marker. The function will then iterate over these symbols simultaneously and decide whether a plants genotype is a recombinant. tra. The recombinants will be summed at the end and the calculation will follow as explained above. The result is the recombination factor, which will be returned as a value where this function is 'called'.


In [17]:

def calculateRF(ref_marker, compare_marker):
    
    # set a variable
    recombinant_count = 0
    unused_count = 0 # -'s
    # iterate over both lists simultaneously whilst comparing values
    # 
    for ref_genotype, comp_genotype in zip(ref_marker, compare_marker):
        
        ## UNCERTAIN IS IT Ref == COMP OR REF != COMP OR REF = B COMP = A??
        ## 
        if (ref_genotype == 'b' and comp_genotype == 'a') or (
            comp_genotype == 'b' and  ref_genotype == 'a'):
            recombinant_count += 1
        if ref_genotype == '-' or comp_genotype == '-':
            unused_count += 1
    
    return recombinant_count / (len(ref_marker) - unused_count)


# Creating the RF matrix
Now, using a nested foreach loop, we can first iterate over each key (a marker, lets call it the reference marker) in our dataset, and then iterate again over each marker (lets call this the compare_marker), creating 144\*144 instances to compare markers in. This is redundant, because each marker will occur in both loops, so marker_A will be compared to marker_B at first, and later marker_B will also be compared to marker_A. I couldn't be bothered to figure out a cleaner, more resource-friendly way of doing this (without checking availibilty of each swapped pair in the dictionary we're building).

> In the code, some artifacts of previous versions are left for reference' sake.


In [18]:
# iterate over each key, value pair 

recombinant_factor = dict() # old 
recombinant_factors_try2 = dict()


# reference and compare_to markers.
for ref_marker, ref_genotype in identifier_dict.items():
    # and again, so we form a matrix 
    # NOTE; THIS MATRIX WONT HAVE THEIR AXES NICELY SORTED!!!!!!!!!!!
    recombinant_factors_try2[f"{ref_marker}"] = dict()
    for comp_marker, comp_genotype in identifier_dict.items():
        
        # skip same keys and skip previously already matched keys
        if (not ref_marker == comp_marker):
            recombinant_factor[f"{ref_marker}_VS_{comp_marker}"]  = calculateRF(ref_genotype, comp_genotype)

            recombinant_factors_try2[f"{ref_marker}"][f"{comp_marker}"] = calculateRF(ref_genotype, comp_genotype)

        

Example of our nested dictionaries:

In [19]:
"""old                                            
for i, (k, v) in enumerate(recombinant_factor.items()):
    if i > 10:
        break
    
    print(k,v)
print(len(recombinant_factor))
"""

for i, (k, v) in enumerate(recombinant_factors_try2.items()):
    if i > 10:
        break
    
    print(k,list(v.items())[:2])
print(len(recombinant_factors_try2))


PVV4_1 [('CRY2_2', 0.5925925925925926), ('AXR-1_3', 0.6024844720496895)]
CRY2_2 [('PVV4_1', 0.5925925925925926), ('AXR-1_3', 0.5590062111801242)]
AXR-1_3 [('PVV4_1', 0.6024844720496895), ('CRY2_2', 0.5590062111801242)]
HH.335C_4 [('PVV4_1', 0.6234567901234568), ('CRY2_2', 0.5987654320987654)]
DF.162L_5 [('PVV4_1', 0.6358024691358025), ('CRY2_2', 0.6111111111111112)]
BH.147L_6 [('PVV4_1', 0.6481481481481481), ('CRY2_2', 0.6234567901234568)]
EC.480C_7 [('PVV4_1', 0.6604938271604939), ('CRY2_2', 0.6358024691358025)]
CH.160L_11 [('PVV4_1', 0.7391304347826086), ('CRY2_2', 0.7329192546583851)]
AD.121C_12 [('PVV4_1', 0.7453416149068323), ('CRY2_2', 0.7329192546583851)]
AD.106L_13 [('PVV4_1', 0.7453416149068323), ('CRY2_2', 0.7391304347826086)]
GB.112L_14 [('PVV4_1', 0.7654320987654321), ('CRY2_2', 0.7654320987654321)]
81


# Store recombinant factors in a matrix (tsv)


In [20]:
sorted_keys = list(recombinant_factors_try2.keys())
sorted_keys.sort()

factors = recombinant_factors_try2

with open (OUTPUT_FILENAME, 'w') as out:
    # set headers
    out.write("X\t") # topleft cell
    for column_header in sorted_keys:
        out.write(f"{column_header}\t")
    out.write("\n")
    
    # set leftmost cell key/marker-name
    for row_header in sorted_keys:
        out.write(f"{row_header}\t")
        for key in sorted_keys:
            if (not key == row_header):
                recombinant_distance = factors[row_header][key]
                out.write(f"{recombinant_distance}\t")
            else:
                # there's no distance between marker x with itself
                out.write("X\t")
        out.write("\n")


# Visualise the gene distances
Sadly, it's too much of a hassle to get Python's turtle module to work inside jupyter notebook (and display output in it, too), so that this part is done in a serperate `.py` script.

In [14]:
# todo draw markers with turtle.
#from turtle import *

#t = Turtle()
#t.color('red')
#t.shape('turtle')
#t.forward(10)